# Summary

In this notebook we will fit a static dimensionality reduction model (factor analysis, FA) and dynamic dimensionality reduction model (a latent dynamical system, LDS) to some neural data. We will use the same neural data provided for the encoding and decoding portion of the course (courtesy of ? via Josh), which is available here:
https://www.dropbox.com/s/jcief15oql3tkll/example_data_m1.pickle?dl=0 

<br>

First run "easy_install numpy scipy pykalman" in the terminal.

As before, much of the notebook is already filled in, and you should provide code in the empty cells.

## 1. Import Packages

In [ ]:
#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline
import pickle

## 2. Load Data

The dataset is a recording of neurons in motor cortex, while a monkey is reaching to targets. Previously we fit encoding and decoding models between the x/y velocities of the movement and the neural activity (both of which have been put into 50 ms time bins). This time we will ignore the velocities and try to make sense of this data by just using the neural recordings.

"neural_data" is a matrix of size "number of time bins" x "number of neurons", where each entry is the firing rate of a given neuron in a given time bin

"vels_binned" is a matrix of size "number of time bins" x 2, where the columns are the x and y velocities.

First, in the cell below, enter the folder your data is in. For example, if it is in your current directory: <br>
folder=''

In [ ]:
folder = ''

Note that you'll need to change what is commented/uncommented below if you are running python 2

In [ ]:
with open(folder+'example_data_m1.pickle','rb') as f:
    neural_data,vels_binned=pickle.load(f,encoding='latin1') #If using python 3
#     neural_data,vels_binned=pickle.load(f) #If using python 2

We will refer to the neural_data as "Y" (the observations) and we will not use the velocities for model fitting.

In [ ]:
Y=np.copy(neural_data)
X=np.copy(vels_binned)

## 3. Preprocess Data - split into training/testing sets
We will fit the models on the training set, and then check its performance on the testing set.

In [ ]:
#Set what part of data should be part of the training/testing/validation sets
training_range=[0, 0.7]
testing_range=[0.7, 1]

num_examples=X.shape[0]

#Note that each range has a buffer of"bins_before" bins at the beginning, and "bins_after" bins at the end
#This makes it so that the different sets don't include overlapping neural data
training_set=np.arange(np.int(np.round(training_range[0]*num_examples)),np.int(np.round(training_range[1]*num_examples)))
testing_set=np.arange(np.int(np.round(testing_range[0]*num_examples)),np.int(np.round(testing_range[1]*num_examples)))

#Get training data
Y_train=Y[training_set,:]
X_train=X[training_set,:]

#Get testing data
Y_test=Y[testing_set,:]
X_test=X[testing_set,:]

## 4. Static dimensionality reduction - Factor Analysis (FA)

In [ ]:
#Import factor analysis function (isn't this easy and fun??)
from sklearn.decomposition import FactorAnalysis as FA
from sklearn.decomposition import PCA

Start by fitting a model with two components. The FA documentation can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FactorAnalysis.html).

Next, plot the observations from the training into the 2D latent space found by your model.

Do two components (latent variables) actually fit the data well? As mentioned in class, one of the difficulties of using dimensionality reduction methods is deciding how many components to use. Let's loop over the number of components from 1 to your favorite value of N (maybe start with a small favorite value of N, like 10), calculating the average log-likelihood of both the training samples and testing samples (using FA.score).

Plot the log-likelihoods as a function of number of components.

One commonly seen feature of neural activity is that the variance increases with the mean firing rate. Variance is typically estimated by using multiple repeats of the same trial; however, we don't have that type of information with this data. FA allows us to estimate the variance of the firing rate without repeats of the same trial (something we don't get from PCA).
<br>

Using your favorite number of components given the results above, fit FA, and plot the estimated variance (found in your_fa_model.noise_variance_) as a function of the mean.

We mentioned in class that FA is preferrable over PPCA for fitting neural data because PPCA makes the assumption that the noise variance of each neuron is identical. However, you (should have seen above) that this is not the case with this data. Let's compare FA and PPCA to see if FA is indeed a better model.
<br>

To do so we're going to use a model selection function that comes in scikitlearn called `cross_val_score`, which will do the dirty work of splitting the data into train/test folds, fitting the desired models, _and_ returns cross-validated log-likelihoods (which is calculated for PCA using the PPCA likelihood function; see PCA.noise_variance_ in the scikitlearn documentation for more info).
<br>

The following useful function is adapted from [this scikitlearn example](https://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_vs_fa_model_selection.html#sphx-glr-auto-examples-decomposition-plot-pca-vs-fa-model-selection-py).

In [ ]:
# compare FA and PCA
from sklearn.model_selection import cross_val_score
def compute_scores(Y, max_num_components):
    pca = PCA(svd_solver='full')
    fa = FA()

    pca_scores, fa_scores = [], []
    for n in range(max_num_components):
        print('fitting %i components' % (n + 1))
        pca.n_components = n + 1
        fa.n_components = n
        pca_scores.append(np.mean(cross_val_score(pca, Y, cv=5)))
        fa_scores.append(np.mean(cross_val_score(fa, Y, cv=5)))

    return pca_scores, fa_scores

Call the `compute_scores` function for a given number of components, and plot the resulting scores (log-likelihoods). This may take a minute or two.

## 5. Dynamic dimensionality reduction - LDS

 We are going to run our inference of linear dynamical systems using a python package called pykalman, which implements EM for LDS. 
 <br>
 
 Make sure you have run "easy_install numpy scipy pykalman" in terminal. You may have to reload this notebook after having done that. See [here](http://pykalman.github.io/) for documentation. The documentation goes over a lot of practical tidbits about the estimation of the relevant parameters - you should definitely browse through it.

## Useful Functions

In [ ]:
# Function for calculating likelihoods - take a look if interested
from scipy.stats import multivariate_normal
def compute_llh(kf,observations):
    (filtered_state_means, filtered_state_covariances) = kf.smooth(observations)
    n_timesteps = observations.shape[0]
    loglikelihoods = np.zeros(n_timesteps)
    for t in range(n_timesteps):
        observation = observations[t]
        observation_matrix = kf.observation_matrices
        observation_offset = kf.observation_offsets
        observation_covariance = kf.observation_covariance
        predicted_state_mean = filtered_state_means[t]
        predicted_state_covariance = filtered_state_covariances[t]
        
        predicted_observation_mean = np.dot(observation_matrix,predicted_state_mean).T + observation_offset
        predicted_observation_covariance = (
            np.dot(observation_matrix,
                   np.dot(predicted_state_covariance,
                          observation_matrix.T))
            + observation_covariance
        )

        loglikelihoods[t] = np.log(multivariate_normal.pdf(
            observation,
            mean=predicted_observation_mean,
            cov=predicted_observation_covariance
        ))
    return sum(loglikelihoods)

In [ ]:
# Function for fitting LDS and predict the state and smoothed observations
# The docs may be useful here: http://pykalman.github.io/. Feel free to sneak a peak under the hood - it is exactly as discussed in lecture.

def fit_LDS(Y_train,Y_test,latentdim=2,n_iter=5,smoothorfilt='smooth'):
    # Initialize a Kalman Filter class, while specifying the number of latent dimensions you want.
    kf = KalmanFilter(initial_state_mean=np.zeros((latentdim)), n_dim_obs=Y_train.shape[1]) ##
    # Estimate the states and the transition + observations matrices using EM, while specifying the number of iterations.
    kf.em(Y_train, n_iter=n_iter) ##
    
    llh={}; pred_state={}; smoothed_obs={};
    llh['train'] = compute_llh(kf,Y_train)
    llh['test'] = compute_llh(kf,Y_test)
    
    if smoothorfilt=='smooth':
        # Obtain the predicted state mean and covariances using a kalman *smoother*, for both train and test data.
        # Store the predicted state mean in pred_state['train'] and pred_state['test'] for train and test data.
        (pred_state['train'], filtered_state_covariances) = kf.smooth(Y_train) ##
        (pred_state['test'], smoothed_state_covariances) = kf.smooth(Y_test) ##
    elif smoothorfilt=='filt':
        # Obtain the predicted state mean and covariances using a kalman *filter*, for both train and test data.
        # Store the predicted state mean in pred_state['train'] and pred_state['test'] for train and test data.
        (pred_state['train'], filtered_state_covariances) = kf.filter(Y_train) ##
        (pred_state['test'], smoothed_state_covariances) = kf.filter(Y_test) ##

    # Obtain the mean smoothed observations using the LDS forward / generative model
    # Store the mean smoothed observations in smoothed_obs['train'] and smoothed_obs['test'] for train and test data.
    smoothed_obs['train'] = np.dot(kf.observation_matrices,pred_state['train'].T).T + kf.observation_offsets ##
    smoothed_obs['test'] = np.dot(kf.observation_matrices,pred_state['test'].T).T + kf.observation_offsets ##

    return (llh,pred_state,smoothed_obs)

In [ ]:
# Fits a linear decoding model from Y_train to X_train based on the training data, and returns the R^2 on test data. 
def fit_linear_model(Y_train,X_train,Y_test,X_test):
    reg = sklearn.linear_model.LinearRegression() ##
    reg.fit(Y_train, X_train) ## 
    return reg.score(Y_test, X_test) ##


## Simulated Data

In [ ]:
from pykalman import KalmanFilter

# This is an example of running the kalman filter where, knowing the transition and observation matrices, 
# you estimate the state. Take a look if you want to see an example and play around with transition and
# observation matrices, but it is not important for this tutorial.
kf = KalmanFilter(transition_matrices = [[1, 1], [0, 1]], observation_matrices = [[0.1, 0.5], [-0.3, 0.0]])
measurements = np.asarray([[1,0], [0,0], [0,1]])
kf = kf.em(measurements, n_iter=15)
llh = compute_llh(kf,measurements)
(filtered_state_means, filtered_state_covariances) = kf.filter(measurements)
(smoothed_state_means, smoothed_state_covariances) = kf.smooth(measurements)

filtered_measurements=np.dot(kf.observation_matrices,filtered_state_means.T).T + kf.observation_offsets
smoothed_measurements=np.dot(kf.observation_matrices,smoothed_state_means.T).T + kf.observation_offsets

plt.figure()
plt.plot(measurements[:,0],measurements[:,1],'.', label='Observations')
plt.plot(filtered_measurements[:,0],filtered_measurements[:,1],'.-', label='Filtered Observations')
plt.plot(smoothed_measurements[:,0],smoothed_measurements[:,1],'.-', label='Smoothed Observations')
plt.legend()
plt.title('llh='+str(llh))
plt.show()

In [ ]:
# This is an example of running the EM algorithm where you first estimate the transition and observation matrices,
# then you estimate the state.
n_iter=5

# Consider one latent dimension
llh_l1,pred_state,smoothed_obs_l1=fit_LDS(measurements,measurements,latentdim=1,n_iter=n_iter)
plt.figure()
plt.plot(measurements[:,0],measurements[:,1],'.-', label='Observations')
plt.plot(smoothed_obs_l1['train'][:,0],smoothed_obs_l1['train'][:,1],'.-', label='Smoothed Observations')
plt.legend()
plt.title('Model with 1 latent, llh='+str(llh_l1['train']))
plt.show()

# Consider two latent dimensions. Remember, our observations are dimension 2.
llh_l2,pred_state,smoothed_obs_l2=fit_LDS(measurements,measurements,latentdim=2,n_iter=n_iter)
plt.figure()
plt.plot(measurements[:,0],measurements[:,1],'.-', label='Observations')
plt.plot(smoothed_obs_l2['train'][:,0],smoothed_obs_l2['train'][:,1],'.-', label='Smoothed Observations')
plt.legend()
plt.title('Model with 2 latents, llh='+str(llh_l2['train']))
plt.show()

# Let's iterate over the number of latents to get an idea of what's happening as we increase this.
llh_latents=[fit_LDS(measurements,measurements,latentdim=i,n_iter=n_iter)[0]['train'] for i in np.arange(10)]
plt.figure()
plt.plot(np.arange(10),llh_latents,'-o')
plt.xlabel('Number of components')
plt.ylabel('Log-likelihood')
plt.show()


*Question: Why does the model with one latent do so poorly? What happens after number of latents > number of observations, and why?*
<br>

*Question: What happens as you increase the number of iterations? Why? (Read the docs if you want to find out more)*

## Neural Data

We will try to get an optimal number of latent states for LDS. We will base our decision on two factors (a) a plateau or decrease in test R^2, (b) ability to decode the velocity from the latents. 
<br>

*Question: Can the test R^2 theoretically decrease for LDS as the latent dimension increases? How about PCA and FA?*

<br>
We subsample the neurons and timepoints because EM takes a long time to run. Feel free to try higher numbers of neurons if needed.

In [ ]:
obsdim=20; maxT=1000
neurons=np.random.choice(Y_train.shape[1], obsdim, replace=False)

Y_train_subset=Y_train[:maxT,neurons]
Y_test_subset=Y_test[:,neurons]

# get rid of silent neurons
silent_indxs = np.where(np.sum(Y_train_subset, axis=0)==0)[0]
Y_train_subset=np.delete(Y_train_subset, silent_indxs, 1)
Y_test_subset=np.delete(Y_test_subset, silent_indxs, 1)

print(Y_train_subset.shape); print(Y_test_subset.shape)


We loop over the number of latent dimension, while storing all our results. Choose a maximum latent dimension and a latent step size.

In [ ]:
from pykalman import KalmanFilter
import sklearn

maxlatentdim=20; latentstep=3; latentvec=np.arange(1,maxlatentdim,latentstep)

trainstate, teststate, trainobs, testobs, trainllh, testllh, trainr2, testr2 = [], [], [], [], [], [], [], []
for latentdim in latentvec:
    print('fitting %i latent dimensions' % latentdim)
    llh,pred_state,smoothed_obs = fit_LDS(Y_train_subset,Y_test_subset,latentdim=latentdim)
    trainstate.append(pred_state['train']); teststate.append(pred_state['test'])
    trainobs.append(smoothed_obs['train']); testobs.append(smoothed_obs['test'])
    trainllh.append(llh['train']); testllh.append(llh['test'])
    trainr2.append(sklearn.metrics.r2_score(Y_train_subset,smoothed_obs['train']))
    testr2.append(sklearn.metrics.r2_score(Y_test_subset,smoothed_obs['test']))

### plot the log likelihood and R^2 as a function of the number of components

Decode the x and y velocities from 

- raw data
- latent states of LDS
- smoothed observations of LDS
- PCA latents
- FA latents
<br>

*Question: Do you expect the decoding accuracy to be the same from the latents and the smoothed observations? Why or why not?*

In [ ]:
X_train_subset=X_train[:maxT,:]
X_test_subset=X_test[:,:]

# for the x and y velocity as outputs
for xdim in np.arange(X_test_subset.shape[1]):
    # fit a linear decoder from all neurons to x/y velocity. Call the test R^2 r2_allobs.
        
    # fit a linear decoder from the subsampled neurons to x/y velocity. Call the test R^2 r2_obs.
    
    
    r2_state, r2_smoothobs, r2_pca, r2_fa = [],[],[],[]
    for latentdim in np.arange(len(trainstate)):
        # fit a linear decoder from the estimated state to x/y velocity. Append the test R^2 to r2_state.
        
        # fit a linear decoder from the smoothed observations to x/y velocity. Append the test R^2 to r2_smoothobs.
        
        # fit a linear decoder from the PCA latents to x/y velocity. Append the test R^2 to r2_state.
        
        # fit a linear decoder from the FA latents to x/y velocity. Append the test R^2 to r2_state.
        
    # plot r2s for subsampled raw data (all/subsampled), LDS state, smoothed latents, PCA and FA compenents
    

*Question: Do you see the decoding from LDS State to be better than from the subsampled raw data at any point? Why do you think this is / is not happening?*

## 6. Additional Notes

Further questions
<br>
(a) How does EM seem to scale with the number of neurons? What would your approach be to speed up the algorithms while recording many more neurons simultaneously?

# Questions?

If you have questions on this notebook, please contact either:<br>
Matt: m.whiteway@columbia.edu <br>
Shreya: ss5513@columbia.edu